# Drivers's performance - Braspress

## Libraries

In [2]:
import importlib
import numpy as np
import pandas as pd
import telemetry
import braspress

## Settings

In [32]:
settings = {
    "telemetry_dir": "Dataset/Telemetry Data",
    "braspress_branches_path": "Dataset/braspress_branches.csv",
    "braspress_valid_lines_path": "Dataset/valid_lines.csv",
    "dates": (
        (2024, 10),
        (2024, 11),
        (2024, 12)
    ),
    "min_displacement": 5.0,
    "braspress_proximity_threshold": 5.0,
    "avg_fuel_consumption_goal_number_of_std": 1.0
}

## Load Data

In [33]:
importlib.reload(telemetry)

<module 'telemetry' from 'f:\\Google Drive Sync\\Projects\\Braspress Drivers Performance\\telemetry.py'>

In [34]:
batches = []

for date in settings["dates"]:
    batches.append(telemetry.Telemetry(settings, date[0], date[1], max_lines_to_read=2000))

In [35]:
driver_stats = pd.concat([batch.get_driver_stats() for batch in batches])
driver_stats

,driver_id,avg_fuel_consumption,avg_fuel_consumption_goal,reach_fuel_goal
0,11276526725,4.266182,5.654330,0.0
0,12058615646,2.645019,2.645019,0.0
0,13312035899,2.757076,NaN,NaN
0,13599656886,4.076034,NaN,NaN
0,15261616808,2.914765,NaN,NaN
...,...,...,...,...
0,91442036753,3.098900,NaN,NaN
0,94144982149,2.668194,2.668194,0.0
0,96089725668,2.933979,NaN,NaN
0,96196130153,3.764166,NaN,NaN


In [37]:
driver_stats.to_excel("driver_fuel_performance.xlsx")

**October 2024 Analysis**

Extract data

In [23]:
importlib.reload(telemetry)
oct2024 = telemetry.Telemetry(settings, 2024, 10, max_lines_to_read=2000)

In [25]:
fleet_drivers_stats = oct2024.get_fleet_driver_groups()
fleet_drivers_stats

,fleet,driver_id,line_id,avg_fuel_consumption
0,281,3347814770,RIOBTA,3.219765
1,2559,13312035899,BTATRL,2.757076
2,21374,1537127756,BCGBTA,3.125225
3,21374,2276154764,SDRV2C,2.483615
4,21374,2276154764,V2CVIX,3.856846
...,...,...,...,...
69,21407,558775080,BTAFLN,3.061392
70,21042,3388104719,BTAR2C,3.634090
71,21042,3388104719,R2CRIO,4.631068
72,21471,60402938534,NBRITB,3.768923


In [26]:
driver_stats = oct2024.get_driver_stats()
driver_stats

,driver_id,avg_fuel_consumption,avg_fuel_consumption_goal,reach_fuel_goal
0,11276526725,4.266182,5.654330,0.0
0,12058615646,2.645019,2.645019,0.0
0,13312035899,2.757076,NaN,NaN
0,13599656886,4.076034,NaN,NaN
0,15261616808,2.914765,NaN,NaN
0,1537127756,3.125225,NaN,NaN
0,17451066851,3.007054,NaN,NaN
0,20421958553,3.466723,3.452120,1.0
0,21266373870,3.107443,NaN,NaN
0,2276154764,3.170231,5.654330,0.0


In [30]:
importlib.reload(braspress)
import braspress

valid_lines_with_stats = braspress.Braspress(settings).extract_valid_lines_with_stats(fleet_drivers_stats)
valid_lines_with_stats.dropna()

,avg_fuel_consumption_goal
line_id,
AJUFSA,3.114934
BCGIPN,2.276272
BCGITB,2.748742
BCGVAG,3.298107
BTABTB,3.375887
BTACWB,2.675488
BTARIO,3.452321
BTBBTA,2.966583
FSAITB,3.452120
